## Spam Mail Data에 대한 이해 

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('./spam.csv',encoding='latin1')

In [ ]:
data[:5]

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
data.info()

In [ ]:
data.isnull().values.any()

In [ ]:
data['v2'].nunique(), data['v1'].nunique()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

data['v1'].value_counts().plot(kind='bar');

In [ ]:
print(data.groupby('v1').size().reset_index(name='count'))

In [ ]:
X_data = data['v2']
y_data = data['v1']
print('메일 본문의 개수: %d' % len(X_data))
print('레이블의 개수: %d' % len(y_data))

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) #5572개의 행을 가진 X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(X_data) #단어를 숫자값, 인덱스로 변환하여 저장

In [ ]:
print(sequences[:5])

In [ ]:
word_index = tokenizer.word_index
print(word_index)

In [ ]:
vocab_size = len(word_index)+1
print('단어 집합의 크기: %d' % (vocab_size))

In [ ]:
n_of_train = int(5572 * 0.8)
n_of_test = int(5572 - n_of_train)
print(n_of_train)
print(n_of_test)

In [ ]:
X_data=sequences
print('메일의 최대 길이 : %d' % max(len(l) for l in X_data))
print('메일의 평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))
plt.hist([len(s) for s in X_data], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

## RNN으로 Spam Mail 분류하기

In [ ]:
from keras.layers import SimpleRNN, Embedding, Dense
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
max_len = 189
# 전체 DataSet의 길이는 189로 맞춘다. (메일의 최대 길이)
data = pad_sequences(X_data, maxlen=max_len)
print("data shape: ", data.shape)

In [ ]:
X_test = data[n_of_train:] #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = y_data[n_of_train:] #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장
X_train = data[:n_of_train] #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = y_data[:n_of_train] #y_data 데이터 중에서 앞의 4457개의 데이터만 저장

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2)

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()